# Классификация на PyTorch Lightning


In [ ]:
%pip install torch pytorch_lightning numpy matplotlib scikit-learn

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [7]:
wine = load_wine()
X = wine.data
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Classes: {len(np.unique(y))}")


Train: torch.Size([142, 13]), Test: torch.Size([36, 13])
Classes: 3


In [8]:
RANDOM_SEED = 1337

class WineDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class WineDataModule(pl.LightningDataModule):
    train_dataset = None
    val_dataset = None
    test_dataset = None
    generator = torch.Generator().manual_seed(RANDOM_SEED)

    def __init__(self, X_train, X_test, y_train, y_test, batch_size=16):
        super().__init__()
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.batch_size = batch_size

        self.train_dataset = None
        self.val_dataset = None
        self.test_dataset = None
    
    def setup(self, stage=None):
        full_dataset = WineDataset(self.X_train, self.y_train)
        
        full_dataset_size = len(full_dataset)
        val_dataset_size = round(full_dataset_size * 0.1)
        train_dataset_size = full_dataset_size - val_dataset_size

        self.train_dataset, self.val_dataset = torch.utils.data.random_split(
            full_dataset, [train_dataset_size, val_dataset_size], 
            self.generator
        )

        self.test_dataset = WineDataset(self.X_test, self.y_test)
    
    def train_dataloader(self):
        if not self.train_dataset:
            raise Exception('Training dataset is not set, run `setup` method first')

        return DataLoader(
            self.train_dataset, 
            batch_size=self.batch_size, 
            shuffle=True, 
            generator=self.generator
        )
    
    def val_dataloader(self):
        if not self.val_dataset:
            raise Exception('Validation dataset is not set, run `setup` method first')

        return DataLoader(
            self.val_dataset, 
            batch_size=self.batch_size, 
            shuffle=True, 
            generator=self.generator
        )
    
    def test_dataloader(self):
        if not self.test_dataset:
            raise Exception('Test dataset is not set, run `setup` method first')

        return DataLoader(
            self.test_dataset, 
            batch_size=self.batch_size, 
            shuffle=True, 
            generator=self.generator
        )

In [9]:
class NeuralNetLightning(pl.LightningModule):
    def __init__(self, input_size, num_classes, lr=0.001):
        super().__init__()
        self.save_hyperparameters()
        self.lr = lr

        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes)
        )

        self.criterion = nn.CrossEntropyLoss()
    
    def __calc_loss_acc(self, batch, log_name_prefix: str):
        x, y = batch
        logits = self(x)

        loss = self.criterion(logits, y)
        acc = (logits.argmax(dim=1) == y).float().mean()

        self.log(f"{log_name_prefix}_loss", loss, prog_bar=True, on_epoch=True)
        self.log(f"{log_name_prefix}_acc", acc, prog_bar=True, on_epoch=True)

        return (loss, acc)

    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch):
        loss, _ = self.__calc_loss_acc(batch, "train")
        return loss
    
    def validation_step(self, batch):
        self.__calc_loss_acc(batch, "val")
    
    def test_step(self, batch):
        self.__calc_loss_acc(batch, "test")
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)


In [10]:
data_module = WineDataModule(X_train, X_test, y_train, y_test, batch_size=16)
model = NeuralNetLightning(input_size=13, num_classes=3, lr=0.001)

print(model)


NeuralNetLightning(
  (model): Sequential(
    (0): Linear(in_features=13, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=3, bias=True)
  )
  (criterion): CrossEntropyLoss()
)


In [11]:
trainer = pl.Trainer(
    max_epochs=100,
    accelerator='auto',
    devices=1,
    log_every_n_steps=5,
    enable_progress_bar=True
)

trainer.fit(model, data_module)


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/Users/shadiabdelsalam/Documents/education/misis/practice-year2/dl2025/lesson2/homework/lib/python3.14/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name      | Type             | Params | Mode  | FLOPs
------------------------------------------------------------

/Users/shadiabdelsalam/Documents/education/misis/practice-year2/dl2025/lesson2/homework/lib/python3.14/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:485: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/Users/shadiabdelsalam/Documents/education/misis/practice-year2/dl2025/lesson2/homework/lib/python3.14/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/shadiabdelsalam/Documents/education/misis/practice-year2/dl2025/lesson2/homework/lib/python3.14/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argume

Epoch 99: 100%|██████████| 8/8 [00:00<00:00, 249.45it/s, v_num=0, train_loss_step=0.00117, train_acc_step=1.000, val_loss=0.015, val_acc=1.000, train_loss_epoch=0.000716, train_acc_epoch=1.000]  

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 8/8 [00:00<00:00, 222.04it/s, v_num=0, train_loss_step=0.00117, train_acc_step=1.000, val_loss=0.015, val_acc=1.000, train_loss_epoch=0.000716, train_acc_epoch=1.000]


In [12]:
test_result = trainer.test(model, data_module)
print(f"Test Accuracy: {test_result[0]['test_acc']:.4f}")

/Users/shadiabdelsalam/Documents/education/misis/practice-year2/dl2025/lesson2/homework/lib/python3.14/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:485: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/Users/shadiabdelsalam/Documents/education/misis/practice-year2/dl2025/lesson2/homework/lib/python3.14/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:434: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 228.61it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                    1.0
        test_loss          0.000776580476667732
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Test Accuracy: 1.0000


In [13]:
model.eval()
with torch.no_grad():
    logits = model(X_test)
    predicted = torch.argmax(logits, dim=1)
    
from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(y_test, predicted, target_names=wine.target_names))



Classification Report:
              precision    recall  f1-score   support

     class_0       1.00      1.00      1.00        14
     class_1       1.00      1.00      1.00        14
     class_2       1.00      1.00      1.00         8

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36

